In [ ]:
using Pkg.TOML

using Random: seed!
using Revise
using RMMMDNets

# enable_gpu()
# disable_gpu()

;

In [ ]:
rmmmdnets_path = pathof(RMMMDNets) |> splitdir |> first |> splitdir |> first
hyper = TOML.parsefile("$rmmmdnets_path/examples/Hyper.toml")

# dataset = "gaussian"
# dataset = "ring"
dataset = "mnist"
# model_name = "gan"
# model_name = "mmdnet"
model_name = "rmmmdnet"

args_dict = parse_toml(hyper, dataset, model_name)
args = parse_args_dict(args_dict; override=(seed=1, lr=1f-3, Df_h="conv", sigma="0.1,1,10,100",))

###

data = get_data(dataset)

;

In [ ]:
function run_exp(args)    
    seed!(args.seed)

    model = get_model(args, data)
    evaluate(data, model)

    dataloader = DataLoader(data, args.batch_size)
    
    train!(model, args.n_epochs, dataloader)
    evaluate(data, model)
    
    model_fname = save!(model)
    
    return model, model_fname
end

model_trained, model_trained_fname = run_exp(args)

;